In [17]:
import csidata as cid
import numpy as np
import scipy.io as sio
import os
import itertools
from random import shuffle

In [18]:
user = 'ownUser1'
load_dir = f'record/prod_data/csi/{user.lstrip('own').lower()}/'
save_dir = f'DeepSeg/01Data_PreProcess/Data_RawCSIDat/'
generate_multiple_antenna_combinations = True
preselected_antenna_pairs = (3, 7, 8)
antenna_pairs_ids = []

In [19]:
print(antenna_pairs_ids)
if generate_multiple_antenna_combinations:
    antenna_combinations = itertools.combinations(range(0, 9), 3)
    valid_combinations = []
    for e in antenna_combinations:
        # add combinations that contain at least 2 antennas from the manually selected antenna pairs
        if len(set(e) & set(preselected_antenna_pairs)) >= 2:
            e = list(e)
            shuffle(e)
            valid_combinations.append(tuple(e))
    antenna_pairs_ids = valid_combinations
    print(antenna_pairs_ids)
else:
    antenna_pairs_ids = [preselected_antenna_pairs]
user_antenna_names = [f"{user}a{''.join(map(str, e))}" for e in antenna_pairs_ids]

[]
[(7, 0, 3), (3, 0, 8), (7, 8, 0), (3, 7, 1), (8, 3, 1), (8, 1, 7), (7, 3, 2), (2, 8, 3), (2, 8, 7), (4, 7, 3), (8, 3, 4), (5, 7, 3), (8, 5, 3), (7, 6, 3), (3, 6, 8), (8, 3, 7), (8, 7, 4), (7, 5, 8), (8, 7, 6)]


In [20]:
GROUPING_2_CARRIERS = ([-28,-26,-24,-22,-20,-18,-16,-14,-12,-10,-8,-6,-4,-2,-1],[1,3,5,7,9,11,13,15,17,19,21,23,25,27,28])
# the subcarrier with id 0 is a null carrier without data... this shifts the index numbers of the second half of carriers down by 1
subcar_ids = [car + 28 for car in GROUPING_2_CARRIERS[0]] + [car + 28 - 1 for car in GROUPING_2_CARRIERS[1]]

In [21]:
import glob
import re
import pandas as pd
filenames = glob.glob('*_seq-*.raw', root_dir=load_dir)
regex = re.compile(r'.*_seq\-(\w\w)\.raw')

fileinfo = [ (filename, regex.match(filename).group(1)) for filename in filenames]
fileinfo.sort()
print(fileinfo)

df = pd.DataFrame(fileinfo, columns=['filename', 'seq'])

df['seq_index'] = df.groupby('seq').cumcount().add(1)
df['new_filename_suffix'] = '_' + df['seq'] + '_' + df['seq_index'].astype(str) + '.mat'

df

[('2024-10-11T16-53-18-669897_seq-iw.raw', 'iw'), ('2024-10-11T16-54-58-944612_seq-iw.raw', 'iw'), ('2024-10-11T16-56-16-370969_seq-iw.raw', 'iw'), ('2024-10-11T16-57-15-673881_seq-iw.raw', 'iw'), ('2024-10-11T16-59-29-810390_seq-iw.raw', 'iw'), ('2024-10-11T17-02-22-296947_seq-ph.raw', 'ph'), ('2024-10-11T17-04-18-741718_seq-ph.raw', 'ph'), ('2024-10-11T17-05-49-170291_seq-ph.raw', 'ph'), ('2024-10-11T17-06-48-096612_seq-ph.raw', 'ph'), ('2024-10-11T17-09-06-000850_seq-ph.raw', 'ph'), ('2024-10-11T17-11-48-279807_seq-ph.raw', 'ph'), ('2024-10-11T17-14-07-312598_seq-rp.raw', 'rp'), ('2024-10-11T17-16-06-407371_seq-rp.raw', 'rp'), ('2024-10-11T17-17-06-928945_seq-rp.raw', 'rp'), ('2024-10-11T17-18-38-301779_seq-rp.raw', 'rp'), ('2024-10-11T17-19-40-755138_seq-rp.raw', 'rp'), ('2024-10-11T17-20-49-594629_seq-rp.raw', 'rp'), ('2024-10-11T17-24-50-860221_seq-sd.raw', 'sd'), ('2024-10-11T17-27-50-910991_seq-sd.raw', 'sd'), ('2024-10-11T17-28-59-804945_seq-sd.raw', 'sd'), ('2024-10-11T17-30-

,filename,seq,seq_index,new_filename_suffix
0,2024-10-11T16-53-18-669897_seq-iw.raw,iw,1,_iw_1.mat
1,2024-10-11T16-54-58-944612_seq-iw.raw,iw,2,_iw_2.mat
2,2024-10-11T16-56-16-370969_seq-iw.raw,iw,3,_iw_3.mat
3,2024-10-11T16-57-15-673881_seq-iw.raw,iw,4,_iw_4.mat
4,2024-10-11T16-59-29-810390_seq-iw.raw,iw,5,_iw_5.mat
5,2024-10-11T17-02-22-296947_seq-ph.raw,ph,1,_ph_1.mat
6,2024-10-11T17-04-18-741718_seq-ph.raw,ph,2,_ph_2.mat
7,2024-10-11T17-05-49-170291_seq-ph.raw,ph,3,_ph_3.mat
8,2024-10-11T17-06-48-096612_seq-ph.raw,ph,4,_ph_4.mat
9,2024-10-11T17-09-06-000850_seq-ph.raw,ph,5,_ph_5.mat


In [22]:
for oldfn, new_fn_suffix in df[['filename', 'new_filename_suffix']].values:
    data = cid.load(os.path.join(load_dir, oldfn))
    csi = data.csi
    print(oldfn, csi.shape)

    for selected_antennas in antenna_pairs_ids:
        idx_rx = [i // 3 for i in selected_antennas]
        idx_tx = [i % 3 for i in selected_antennas]
        print(selected_antennas)
        print(f"{list(zip(idx_rx, idx_tx))}")
        # select the csi values for the selected antenna pairs
        csi_sel = csi[:, idx_rx, idx_tx, :]

        # filter out the subcarriers we are interested in
        csi_one_rx_reduced_car:np.ndarray = csi_sel.take(subcar_ids, axis=2)
        # bring data in format used by DeepSeg, where the csi values for all antennas are stored in a single dimension
        reshaped = csi_one_rx_reduced_car.reshape(-1, 90)
        # store the csi_one_rx_reduced_car in a new '.mat' file using scipy.io.savemat
        user_name = f"{user}a{''.join(map(str, selected_antennas))}"
        # print(user_name)
        os.makedirs(os.path.join(save_dir, user_name), exist_ok=True)
        sio.savemat(os.path.join(save_dir, user_name, user_name + new_fn_suffix), {'data': reshaped})
        print(f"Saved {os.path.join(save_dir, user_name, user_name + new_fn_suffix)}")


2024-10-11T16-53-18-669897_seq-iw.raw (40888, 3, 3, 56)
(7, 0, 3)
[(2, 1), (0, 0), (1, 0)]
Saved DeepSeg/01Data_PreProcess/Data_RawCSIDat/ownUser1a703/ownUser1a703_iw_1.mat
(3, 0, 8)
[(1, 0), (0, 0), (2, 2)]
Saved DeepSeg/01Data_PreProcess/Data_RawCSIDat/ownUser1a308/ownUser1a308_iw_1.mat
(7, 8, 0)
[(2, 1), (2, 2), (0, 0)]
Saved DeepSeg/01Data_PreProcess/Data_RawCSIDat/ownUser1a780/ownUser1a780_iw_1.mat
(3, 7, 1)
[(1, 0), (2, 1), (0, 1)]
Saved DeepSeg/01Data_PreProcess/Data_RawCSIDat/ownUser1a371/ownUser1a371_iw_1.mat
(8, 3, 1)
[(2, 2), (1, 0), (0, 1)]
Saved DeepSeg/01Data_PreProcess/Data_RawCSIDat/ownUser1a831/ownUser1a831_iw_1.mat
(8, 1, 7)
[(2, 2), (0, 1), (2, 1)]
Saved DeepSeg/01Data_PreProcess/Data_RawCSIDat/ownUser1a817/ownUser1a817_iw_1.mat
(7, 3, 2)
[(2, 1), (1, 0), (0, 2)]
Saved DeepSeg/01Data_PreProcess/Data_RawCSIDat/ownUser1a732/ownUser1a732_iw_1.mat
(2, 8, 3)
[(0, 2), (2, 2), (1, 0)]
Saved DeepSeg/01Data_PreProcess/Data_RawCSIDat/ownUser1a283/ownUser1a283_iw_1.mat
(2, 8, 7

# Generate Label File

In [23]:
name_dict = df[['filename', 'new_filename_suffix']].set_index('filename').to_dict()['new_filename_suffix']
name_dict

{'2024-10-11T16-53-18-669897_seq-iw.raw': '_iw_1.mat',
 '2024-10-11T16-54-58-944612_seq-iw.raw': '_iw_2.mat',
 '2024-10-11T16-56-16-370969_seq-iw.raw': '_iw_3.mat',
 '2024-10-11T16-57-15-673881_seq-iw.raw': '_iw_4.mat',
 '2024-10-11T16-59-29-810390_seq-iw.raw': '_iw_5.mat',
 '2024-10-11T17-02-22-296947_seq-ph.raw': '_ph_1.mat',
 '2024-10-11T17-04-18-741718_seq-ph.raw': '_ph_2.mat',
 '2024-10-11T17-05-49-170291_seq-ph.raw': '_ph_3.mat',
 '2024-10-11T17-06-48-096612_seq-ph.raw': '_ph_4.mat',
 '2024-10-11T17-09-06-000850_seq-ph.raw': '_ph_5.mat',
 '2024-10-11T17-11-48-279807_seq-ph.raw': '_ph_6.mat',
 '2024-10-11T17-14-07-312598_seq-rp.raw': '_rp_1.mat',
 '2024-10-11T17-16-06-407371_seq-rp.raw': '_rp_2.mat',
 '2024-10-11T17-17-06-928945_seq-rp.raw': '_rp_3.mat',
 '2024-10-11T17-18-38-301779_seq-rp.raw': '_rp_4.mat',
 '2024-10-11T17-19-40-755138_seq-rp.raw': '_rp_5.mat',
 '2024-10-11T17-20-49-594629_seq-rp.raw': '_rp_6.mat',
 '2024-10-11T17-24-50-860221_seq-sd.raw': '_sd_1.mat',
 '2024-10-

In [24]:
for user_antenna_name in user_antenna_names:
    df_lab = pd.read_csv(f'record/prod_data/{user.lstrip('own').lower()}.csv')
    df_lab['fileName'] = df_lab['fileName'].map(lambda fn: user_antenna_name + name_dict[fn + '.raw'].rstrip('.mat'))
    save_dir = f'DeepSeg/02Segment_ExtractTrainData/Label_CsiAmplitudeCut/{user_antenna_name}'
    print(user_antenna_name, df_lab.loc[0, 'fileName'], os.path.realpath(save_dir))
    os.makedirs(save_dir, exist_ok=True)
    df_lab.to_csv(f'{save_dir}/ManualSegment.csv', index=False)

ownUser1a703 ownUser1a703_iw_1 /home/felix/Documents/uni/BaProj/bachelorproject/DeepSeg/02Segment_ExtractTrainData/Label_CsiAmplitudeCut/ownUser1a703
ownUser1a308 ownUser1a308_iw_1 /home/felix/Documents/uni/BaProj/bachelorproject/DeepSeg/02Segment_ExtractTrainData/Label_CsiAmplitudeCut/ownUser1a308
ownUser1a780 ownUser1a780_iw_1 /home/felix/Documents/uni/BaProj/bachelorproject/DeepSeg/02Segment_ExtractTrainData/Label_CsiAmplitudeCut/ownUser1a780
ownUser1a371 ownUser1a371_iw_1 /home/felix/Documents/uni/BaProj/bachelorproject/DeepSeg/02Segment_ExtractTrainData/Label_CsiAmplitudeCut/ownUser1a371
ownUser1a831 ownUser1a831_iw_1 /home/felix/Documents/uni/BaProj/bachelorproject/DeepSeg/02Segment_ExtractTrainData/Label_CsiAmplitudeCut/ownUser1a831
ownUser1a817 ownUser1a817_iw_1 /home/felix/Documents/uni/BaProj/bachelorproject/DeepSeg/02Segment_ExtractTrainData/Label_CsiAmplitudeCut/ownUser1a817
ownUser1a732 ownUser1a732_iw_1 /home/felix/Documents/uni/BaProj/bachelorproject/DeepSeg/02Segment_Ex

In [25]:
user_antenna_names

['ownUser1a703',
 'ownUser1a308',
 'ownUser1a780',
 'ownUser1a371',
 'ownUser1a831',
 'ownUser1a817',
 'ownUser1a732',
 'ownUser1a283',
 'ownUser1a287',
 'ownUser1a473',
 'ownUser1a834',
 'ownUser1a573',
 'ownUser1a853',
 'ownUser1a763',
 'ownUser1a368',
 'ownUser1a837',
 'ownUser1a874',
 'ownUser1a758',
 'ownUser1a876']